In [1]:
import open3d as o3d
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from neurals.network import PCN

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


pybullet build time: May 20 2022 19:44:17


In [3]:
pcds = []
pcds.append(o3d.io.read_point_cloud("../data/seeds/pointclouds/pose_0_pcd.ply"))
pcds.append(o3d.io.read_point_cloud("../data/seeds/pointclouds/pose_1_pcd.ply"))
pcds.append(o3d.io.read_point_cloud("../data/seeds/pointclouds/pose_2_pcd.ply"))
pcds.append(o3d.io.read_point_cloud("../data/seeds/pointclouds/pose_3_pcd.ply"))
pcds.append(o3d.io.read_point_cloud("../data/seeds/pointclouds/pose_4_pcd.ply"))
pcds.append(o3d.io.read_point_cloud("../data/seeds/pointclouds/pose_5_pcd.ply"))
dfs = []
dfs.append(np.load("../data/seeds/pointclouds_df/pose_0_pcd.npy"))
dfs.append(np.load("../data/seeds/pointclouds_df/pose_1_pcd.npy"))
dfs.append(np.load("../data/seeds/pointclouds_df/pose_2_pcd.npy"))
dfs.append(np.load("../data/seeds/pointclouds_df/pose_3_pcd.npy"))
dfs.append(np.load("../data/seeds/pointclouds_df/pose_4_pcd.npy"))
dfs.append(np.load("../data/seeds/pointclouds_df/pose_5_pcd.npy"))

In [4]:
points_list = []
dfs_list = []
for pcd, df in zip(pcds, dfs):
    for i in range(100):
        points = np.asarray(pcd.points)
        points_index = np.random.choice(len(points), 1024, replace=False)
        points = points[points_index]
        points_list.append(points)
        dfs_list.append(df[points_index])
print(len(points_list), len(dfs_list))

600 600


In [10]:
class TestDataset(Dataset):
    def __init__(self, points_list, dfs_list):
        self.pcds = np.asarray(points_list)
        self.dfs = np.asarray(dfs_list)
        self.mapping = []
        for i in range(6):
            self.mapping += [i] * 100
        self.mapping = np.array(self.mapping).astype(np.int64)

    def __len__(self):
        return len(self.mapping)

    def __getitem__(self, index):
        mapping = self.mapping[index]
        pcd = self.pcds[index]
        df = self.dfs[index]
        ret_pcd = pcd + np.random.normal(size=pcd.shape, scale = 0.005)
        ret_df = df + np.random.normal(size=df.shape, scale=0.005)
        return mapping, ret_pcd, ret_df

dataset = TestDataset(points_list, dfs_list)
(training_dataset, test_dataset) = \
        torch.utils.data.random_split(
            dataset, (550, 50))

In [11]:
class TestNN(nn.Module):
    def __init__(self):
        super(TestNN, self).__init__()
        self.pcn = PCN(4,latent_dim=20)
        #self.pcn = PointNet2(feat_dim=0)
        self.fc1 = nn.Linear(20, 64)
        self.out = nn.Linear(64, 6)

    def forward(self, pcd, df):
        df = df.view([df.shape[0], df.shape[1],1])
        pcd = torch.cat([pcd, df], axis=2)
        latent = self.pcn(pcd)
        x = self.fc1(latent).relu()
        x = torch.nn.functional.log_softmax(self.out(x), dim=1)
        return x
net = TestNN().cuda()

In [14]:
optim = torch.optim.Adam(net.parameters(), lr=1e-3)
loss_fn = torch.nn.NLLLoss()
dataloader = DataLoader(training_dataset, batch_size=64)

In [15]:
for i in range(500):
    batch_cnt = 0
    total_loss = 0
    for label, pointcloud, df in dataloader:
        optim.zero_grad()
        label = label.cuda()
        pointcloud = pointcloud.cuda().float()
        df = df.cuda().float()
        pred = net(pointcloud, df)
        loss = loss_fn(pred, label)
        loss.backward()
        optim.step()
        batch_cnt += 1
        total_loss += float(loss)
    print(f"Total Loss: {total_loss/batch_cnt}")

Total Loss: 1.7964346806208293
Total Loss: 1.7920001745224
Total Loss: 1.7902297841178045
Total Loss: 1.7878453466627333
Total Loss: 1.7840404907862346
Total Loss: 1.7761289013756647
Total Loss: 1.7575562927458022
Total Loss: 1.7158650954564412
Total Loss: 1.6288912296295166
Total Loss: 1.471411108970642
Total Loss: 1.2505370643403795
Total Loss: 1.030300219853719
Total Loss: 0.8030530876583524
Total Loss: 0.5726111829280853
Total Loss: 0.39522971709569293
Total Loss: 0.2851282109816869
Total Loss: 0.2144550151295132
Total Loss: 0.1579073957271046
Total Loss: 0.10997884058290058
Total Loss: 0.07510660505957073
Total Loss: 0.04472229422794448
Total Loss: 0.0342108156118128
Total Loss: 0.02036127344601684
Total Loss: 0.014587666114999188
Total Loss: 0.01177131746792131
Total Loss: 0.008796499628159735
Total Loss: 0.007127902884450223
Total Loss: 0.006618354055616591
Total Loss: 0.005758540032224523
Total Loss: 0.004540259535941813
Total Loss: 0.0034265331147859492
Total Loss: 0.003061506

In [17]:
dataloader = DataLoader(test_dataset, batch_size=1)
batch_cnt = 0
total_loss = 0
net.eval()
labels = []
for label, pointcloud, df in dataloader:
    label = label.cuda()
    pointcloud = pointcloud.cuda().float()
    df = df.cuda().float()
    pred = net(pointcloud, df)
    labels.append(label)
    loss = loss_fn(pred, label)
    batch_cnt += 1
    total_loss += float(loss)
print(f"Total Test Loss: {total_loss/batch_cnt}")

Total Test Loss: 1.5425620058806544e-06


In [18]:
torch.save(net.pcn.state_dict(), "../neurals/pcn_model/pcn_df_20.pth")

In [19]:
o3d.visualization.draw_geometries([o3d.geometry.PointCloud(o3d.utility.Vector3dVector(training_dataset[10][1]))])

In [15]:
test_dataset[10][1]

array([[-0.04219856, -0.01532705,  0.03520654],
       [-0.03941568,  0.10910279, -0.05009749],
       [ 0.03348964, -0.15582641,  0.04298963],
       ...,
       [-0.20387647, -0.08532091,  0.06352441],
       [-0.06860191,  0.14903795,  0.05420021],
       [ 0.10703852, -0.02935053,  0.06121993]])